# Notes:

## Module 5.1 : Intro to ML Modeling

We have learnt how to run and track ML experiments, and deploy the chosen models into production. The prediction services are now up and running, and generating predictions for given data.

So are we done now?

No, not yet. With time the business concept changes and so is the data. We need to be cognizant about the model performance from time to time so as to take timely appropriate action.

Monitoring ML models is mostly around monitoring four sectors:

- Service Health: General Software health check
- Model Performance: Depending on metrics for the problem
- Data Quality and integrity
- Data Drift & Concept Drift

Over time, ML models may degrade. This is due to one of two effects:

- Data Drift: In which new input data is no longer represented by the model's training dataset. Example: 3 new popular venues were opened in the last month, our Taxi duration model hasn't got samples of this new data in its training dataset

- Concept Drift: In which the concept changes, i.e: The relationship between inputs and outputs has changed (Not necessarily the data itself however).This drift as the name implies is due to "concepts" (i.e: hidden variables, underpinning hypotheses..etc) changing. Example: Taxi cars have been replaced by newer, faster, nimbler cars. Our model can no longer accurately predict trip durations

### Types of monitoring:

Depending upon the requirements we can go for **online** monitoring where we continuously read input and output data so as to find inconsistencies or we read the stored input and output data periodically to monitor the status that is **batch** monitoring.

Batch monitoring:

- is implemented in most of the production scenarios. 
- Pipelines are orchestrated with tools like Prefect or Airflow, where after some steps in the pipelines, monitoring related calculations are done to generate metrics to determine if data and model are behaving as per the expectation.
- From the stored data, metrices can be calculated and stored in a SQL or NOSQL database and visualizations can be prepared with help of Tableau or Power BI. 
- However, Evidently can help in creating required metrics out of the box and produce the visualizations automatically as well. 

In case of online/real time monitoring Evidently generated metrices can be stored in Prometheus database that very well integrates with Grafana for visualization.

## Module 5.2: Environment Setup

### Creating and activate virtual environment:

1. Go to the correct directory by: `cd "mlops-zoomcamp/05-monitoring"`
2. Activate any existing conda virtual evironment: `. /opt/homebrew/anaconda3/bin/activate && conda activate /opt/homebrew/anaconda3/envs/mlops-zoomcamp-venv;`
3. Create a new virtual environment: `conda create -n py11 python=3.11`
4. Activate it by `conda activate py11`
5. Install the required packages by doing `pip install -r requirements.txt`

### Creating config files:

Docker Compose configuration file is a YAML file where you can list all the services. You can use this file to build and run all the services you had mentioned in the file.

Here is the [link](docker-compose.yml) to the completed docker compose file.

Here is the [link](/mlops-zoomcamp/05-monitoring/config/grafana_datasources.yaml) to the completed grafana data sources file.

Once you create the above 2 files, open the Docker app and then in the command line, run `docker-compose up --build` (in your conda environment).

After running the build command, once you get green messages like below, you can check whether you're able to access the services or not:

<img src="notes-images/docker-compose build.png" width="700"/>

- To access Grafana open localhost:3000 on browser and use admin as both user and password.
- To access Adminer open localhost:8080 on browser

## Module 5.3: Prepare reference and model

We will use the January 2022 green taxi data to train the model and we will create the reference dataset from the validation dataset. The validation dataset wouldn't be used in the model training so we can use it to decide the model quality and if we are happy with the model quality then we can use the validation dataset as the reference data later.

Go through the [baseline_model_nyc_taxi_data jupyter notebook](/mlops-zoomcamp/05-monitoring/baseline_model_nyc_taxi_data.ipynb) to see how we can generate Evidently report.

## Module 5.4: Dummy monitoring

Now, let's try to calculate some dummy metrics, log it into a database and access the data through grafana.

Here is the [link](/mlops-zoomcamp/05-monitoring/dummy_metrics_calculation.py) to the completed dummy metrics calculation while.

Once you create the above the file, open the Docker app and then in the command line, run `docker-compose up --build` (in your conda environment).

Then in another terminal with the `py11` environment activated, run `python dummy_metrics_calculation.py`. Open up Adminer by typing localhost:8080 on your browser and enter the following information:

- System: PostgreSQL
- Username and password: check the dummy_metrics_calculation file
- Database: test

Once you login, click on the `dummy_metrics` table and then on `Select data` and you should see this:

<img src="notes-images/dummy_metrics adminer.png" width="700"/>

Then in another tab, open up Grafana by going to localhost:3000 and enter admin as both username and password.

Create a new dashboard by pressing the Dashboards-> New -> New Dashboard.

Then click on Create a new visualization and select PostgreSQL as your system and you should see this:

<img src="notes-images/empty grafana dummy metrics.png" width="700"/>

Type the below SQL query and change the internal on the top to "Last 5 minutes" and you should see a graph like this:

<img src="notes-images/grafana graph with dummy metrics.png" width="700"/>

You can then press the "Apply" button and you will see your graph pinned in the dashboard.

## Module 5.6: Data quality monitoring

Now that we got a feel of how we can populate postgresql and build a grafana dashboard using dummy metrics, let's pass the actual data drift metrics and see what our dashboard looks like.

Look at the [evidently_metrics_calculation file](/mlops-zoomcamp/05-monitoring/evidently_metrics_calculation.py) which is a modified version of the [dummy_metrics_calculation file](/mlops-zoomcamp/05-monitoring/dummy_metrics_calculation.py) we were working with earlier.

Make sure to have the docker app running and the `docker-compose up --build` command running in a terminal with the `py11` enviroment before proceeding to the next step.

Once the above file is ready, open up a terminal with `py11` environment and run `prefect server start`.

Open up another terminal with `py11` where you will run `prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api` and then `prefect project init` and you shall see the below image:
<img src="notes-images/prefect-docker-config.png" width="700"/>

Hit enter on "docker" and enter the image_name as evidently-prefect and the tag as prefect:

<img src="notes-images/prefect-docker-image-name.png" width="700"/>

Now run `python evidently_metrics_calculation.py`:

<img src="notes-images/evidently_metrics_calculation.png" width="700"/>

When you log into Adminer at localhost:8080 with the credentials mentioned earlier, you'll see that our dummy_metrics table now has updated information:

<img src="notes-images/updated_dummy_metrics.png" width="700"/>

You can now log into Grafana at localhost:3000 and follow the steps mentioned in Module 5.5 to create a new visualization:

<img src="notes-images/grafana evidently metrics.png" width="700"/>

## Module 5.7: Save Grafana Dashboard

To save the grafana dashboard, we will first create a "config" folder and create a [grafana dashboards file](/mlops-zoomcamp/05-monitoring/config/grafana_dashboards.yaml) where we will use a standard configuration format that is used when creating dashboards. We have specified the name of the dashboard and the path to the dashboard files as `/opt/grafana/dashboards`.

So we need to create a dashboards directory and we will create a [data_drift.json](/mlops-zoomcamp/05-monitoring/dashboards/data_drift.json) file in it cause grafana uses json files to store configurations for dashboards and panels. We update the docker-compose.yml file with the path to the dashboards directory.

Then we have to fill the [data_drift.json](/mlops-zoomcamp/05-monitoring/dashboards/data_drift.json) file:

Go the the dashboard settings and click on the JSON Model. Select the text in the JSON Model and paste it in the data_drift.json:

<img src="notes-images/grafana dashboard json.png" width="700"/>

So now when we stop and restart our docker, we will be able to see the same dashboard.

## Module 5.8 : Debugging with test suites and reports

Let's assume that the prediction_drift on 1st Feb,2022 is above the accepted threshold value. We can dig deeper into this to understand it.

Go through the [debugging jupyter noteebok](/mlops-zoomcamp/05-monitoring/debugging_nyc_taxi_data.ipynb)